In [ ]:
import sys, os
sys.path
from os.path import abspath
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import tensor
from torch.utils.data import DataLoader
from torch.optim import Adam
import torch.nn as nn
from torch.optim.lr_scheduler import CyclicLR
import json
import h5py
import random
import yaml
from addict import Dict
import ast

In [ ]:
def PFbeta(labels, predictions, beta, eps=1e-5):
    # eps is a small error term added for numerical stability
    y_true_count = 0
    ctp = 0
    cfp = 0

    for idx in range(len(labels)):
        prediction = min(max(predictions[idx], 0), 1)
        if (labels[idx]):
            y_true_count += 1
            ctp += prediction
        else:
            cfp += prediction

    beta_squared = beta * beta
    c_precision = (ctp + eps) / (ctp + cfp + eps)
    c_recall = (ctp + eps) / (y_true_count + eps)
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return result
    else:
        return 0.

def MovingAvg(v, size):
    avg = np.empty_like(v)
    avg[:size] = np.mean(v[:2*size])
    avg[len(v)-2*size+1:len(v)] = np.mean(v[len(v)-2*size+1:len(v)])
    for i in range(size, len(v) - size):
        avg[i] = np.mean(v[i-size:i+size])
    return avg

def ConfusionMatrix(p, gt):
    p = np.round(np.array(p))
    gt = np.array(gt)
    tp = np.sum((p == 1) & (gt == 1)).astype(int)
    fp = np.sum((p == 1) & (gt == 0)).astype(int)
    tn = np.sum((p == 0) & (gt == 0)).astype(int)
    fn = np.sum((p == 0) & (gt == 1)).astype(int)
    return tp, fp, tn ,fn

def MetricsFromCM(tp, fp, tn, fn, epsilon=1.0e-6):
    tpr = (tp + epsilon)/(tp + fn + epsilon)
    fpr = (fp + epsilon)/(fp + tn + epsilon)
    precision = (tp + epsilon)/(tp + fp + epsilon)
    tnr = (tn + epsilon)/(fp + tn + epsilon)
    return tpr, fpr, precision, tnr
        
def CreateTensorStats(df, labels):
    plabel = labels[0]
    tlabel = labels[1]
    flabel = labels[2]
    true_pos, false_pos, true_neg, false_neg = [], [], [], []
    true_pr, false_pr, precision, true_nr = [], [], [], []
    f1scores = []
    for i in range(len(df.index)):
        preds = eval(df.loc[i, plabel]).cpu().tolist()
        truths = eval(df.loc[i, tlabel]).cpu().tolist()
        f1 = eval(df.loc[i, flabel]).cpu().item()
        df.loc[i, plabel] = preds
        df.loc[i, tlabel] = truths
        f1scores.append(f1)
        tp, fp, tn, fn = ConfusionMatrix(preds, truths)
        tpr, fpr, prec, tnr = MetricsFromCM(tp, fp, tn, fn)
        true_pos.append(tp)
        false_pos.append(fp)
        true_neg.append(tn)
        false_neg.append(fn)
        true_pr.append(tpr)
        false_pr.append(fpr)
        precision.append(prec)
        true_nr.append(tnr)
    df[flabel] = f1scores
    df["tp"] = true_pos
    df["fp"] = false_pos
    df["tn"] = true_neg
    df["fn"] = false_neg
    df["tpr"] = true_pr
    df["fpr"] = false_pr
    df["precision"] = precision
    df["tnr"] = true_nr
    return

def CreateStats(df, labels, idx):
    plabel = labels[0]
    tlabel = labels[1]
    true_pos, false_pos, true_neg, false_neg = [], [], [], []
    true_pr, false_pr, precision, true_nr = [], [], [], []
    for i in range(len(df.index)):
        preds = ast.literal_eval(df.loc[i, plabel])
        truths = ast.literal_eval(df.loc[i, tlabel])
        tp, fp, tn, fn = ConfusionMatrix(np.asarray(preds)[:, idx],
                                         np.asarray(truths)[:, idx])
        tpr, fpr, prec, tnr = MetricsFromCM(tp, fp, tn, fn)
        true_pos.append(tp)
        false_pos.append(fp)
        true_neg.append(tn)
        false_neg.append(fn)
        true_pr.append(tpr)
        false_pr.append(fpr)
        precision.append(prec)
        true_nr.append(tnr)
    df["tp"] = true_pos
    df["fp"] = false_pos
    df["tn"] = true_neg
    df["fn"] = false_neg
    df["tpr"] = true_pr
    df["fpr"] = false_pr
    df["precision"] = precision
    df["tnr"] = true_nr
    return

def StrListColumnToArray(df: pd.DataFrame, src_column: str):
    return np.array(df.loc[:, src_column].apply(ast.literal_eval).to_list())

def ArrayToColumns(df: pd.DataFrame, arr: np.ndarray, new_columns: list):
    assert arr.shape[1] == len(new_columns)
    return pd.concat((df, pd.DataFrame(arr, columns=new_columns)), axis=1)

def BalancedF1Score(preds, gt, n=None):
    true_ids = np.arange(len(gt))[gt == 1].astype(int)
    false_ids = np.arange(len(gt))[gt == 0].astype(int)
    if n == None:
        n = len(true_ids)
    sel_true_ids = np.random.choice(true_ids, n).astype(int)
    sel_false_ids = np.random.choice(false_ids, n).astype(int)
    sel_gt = np.concatenate((gt[sel_true_ids], gt[sel_false_ids]))
    sel_preds = np.concatenate((preds[sel_true_ids], preds[sel_false_ids]))
    return PFbeta(sel_gt, sel_preds, beta=1.)

def ConvertTensors(df, labels):
    for label in labels:
        for i in range(len(df.index)):
            val = eval(df.loc[i, label]).cpu()
            if len(val) > 1:
                df.loc[i,label] = val.tolist()
            else:
                df.loc[i,label] = val.item()

def Dashboard(report: pd.DataFrame, window_size: int=100):
    loss_ma = MovingAvg(report.loss, window_size)
    bacc_ma = MovingAvg(report.block_accuracy, window_size)
    lr_ma = MovingAvg(report.learning_rate, window_size)
    eacc = report.groupby("epoch").mean(numeric_only=True)[["block_accuracy", "epoch_accuracy"]]
    recall, fpr, precision, tnr = MetricsFromCM(report)
    report["recall"] = recall
    report["fpr"] = fpr
    report["precision"] = precision
    report["tnr"] = tnr
    f1_scores = report.groupby("epoch").mean(numeric_only=True)["f1_scores"]
    fig, axs = plt.subplots(2, 3, figsize=(12,8))
    fig.suptitle("Training Log")
    # plot loss
    axs[0, 0].set_title("loss")
    axs[0, 0].set_xlabel("block")
    axs[0, 0].set_ylabel("loss")
    axs[0, 0].scatter(report.index, report.loss, s=0.3, c="red", label="value")
    axs[0, 0].plot(report.index, loss_ma, "-b", label="moving avg")
    axs[0, 0].legend()
    # plot block acc
    axs[0, 1].set_title("block accuracy")
    axs[0, 1].set_xlabel("block")
    axs[0, 1].set_ylabel("accuracy")
    axs[0, 1].scatter(report.index, report.block_accuracy, s=0.3, c="red")
    axs[0, 1].plot(report.index, bacc_ma, "-b")
    # plot epoch acc vs mean block_acc
    axs[0, 2].set_title("epoch accuracy")
    axs[0, 2].set_xlabel("epoch")
    axs[0, 2].set_ylabel("accuracy")
    axs[0, 2].plot(eacc.index, eacc.epoch_accuracy, "-r",
                   linewidth=0.3, label="eval")
    axs[0, 2].plot(eacc.index, eacc.block_accuracy, "-g",
                   linewidth=0.3, label="train")
    axs[0, 2].legend()
    # plot epoch PFbeta
    axs[1, 0].set_title("F1 Score")
    axs[1, 0].set_xlabel("epoch")
    axs[1, 0].set_ylabel("accuracy")
    axs[1, 0].plot(f1_scores.index, f1_scores, "-r",
                   linewidth=0.3)
    # plot Recall, Precision and TNR
    axs[1, 1].set_title("AUC")
    axs[1, 1].set_xlabel("1 - Specificity (FPR)")
    axs[1, 1].set_ylabel("Sensitivity (TPR)")
    axs[1, 1].plot(fpr, recall, "-r",
                   linewidth=1.)
    axs[1, 1].plot([0., 1.], [0., 1.], "-k", linewidth=0.5)
    # plot learning rate
    axs[1, 2].set_title("Learning Rate")
    axs[1, 2].set_xlabel("block")
    axs[1, 2].set_ylabel("accuracy")
    axs[1, 2].scatter(report.index, report.learning_rate, s=0.3, c="red")
    axs[1, 2].plot(report.index, lr_ma, "-b")

    plt.tight_layout()
    plt.show()
    
def Dashboard2(train_report: pd.DataFrame, eval_report, window_size: int=100):
    CreateStats(train_report, ["predictions", "truths"], 0)
    CreateStats(eval_report, ["predictions", "truths"], 0)
    train_f1 = StrListColumnToArray(train_report, "f1_scores")
    eval_f1 = StrListColumnToArray(eval_report, "f1_scores")
    train_report = ArrayToColumns(train_report, train_f1, ["cancer_f1", "lat_f1", "view_f1", "implant_f1"])
    eval_report = ArrayToColumns(eval_report, eval_f1, ["cancer_f1", "lat_f1", "view_f1", "implant_f1"])
    loss_ma = MovingAvg(train_report.loss, window_size)
    bf1_ma = MovingAvg(train_report.cancer_f1, window_size)
    lr_ma = MovingAvg(train_report.learning_rate, window_size)
    tpr_ma = MovingAvg(train_report.tpr, window_size)
    tnr_ma = MovingAvg(train_report.tnr, window_size)
    sel_cols = ["learning_rate", "tpr", "fpr", "precision", "tnr", "cancer_f1", "lat_f1", 
                "view_f1", "implant_f1"]
    train_epoch_stats = train_report.groupby("epoch").mean(numeric_only=True)[sel_cols]
    balf1scores = []
    balf1latscores = []
    for i in range(len(eval_report.index)):
        preds = np.array(ast.literal_eval(eval_report.loc[i, "predictions"]))
        gt = np.array(ast.literal_eval(eval_report.loc[i, "truths"]))
        balf1scores.append(BalancedF1Score(preds[:, 0], gt[:, 0]))
        balf1latscores.append(BalancedF1Score(preds[:, 1], gt[:, 1]))
    lim = len(eval_report.index)
    epoch_ticks = eval_report.epoch.astype(np.int16)
    fig, axs = plt.subplots(2, 3, figsize=(12,8))
    fig.suptitle("Training Log")
    # plot loss
    axs[0, 0].set_title("Loss")
    axs[0, 0].set_xlabel("Block")
    axs[0, 0].set_ylabel("Loss")
    axs[0, 0].scatter(train_report.index, train_report.loss, s=0.3, c="red", label="value")
    axs[0, 0].plot(train_report.index, loss_ma, "-b", label="moving avg")
    axs[0, 0].legend()
    # plot block stats
    axs[0, 1].set_title("Block Statistics")
    axs[0, 1].set_xlabel("Block")
    axs[0, 1].set_ylabel("Score")
    axs[0, 1].set_ylim([-0.05, 1.05])
    axs[0, 1].scatter(train_report.index, train_report.cancer_f1, s=0.5, c="red", label="cancer_f1_score")
    axs[0, 1].scatter(train_report.index, train_report.tpr, marker="*", s=0.5, c="blue", label="true_pos_rate")
    axs[0, 1].scatter(train_report.index, train_report.tnr, marker="^", s=0.5, c="green", label="true_neg_rate")
    axs[0, 1].plot(train_report.index, bf1_ma, "-c")
    axs[0, 1].plot(train_report.index, tpr_ma, "-y")
    axs[0, 1].plot(train_report.index, tnr_ma, "-m")
    axs[0, 1].legend()
    # plot AUC
    axs[0, 2].set_title("Training AUC")
    axs[0, 2].set_xlabel("1 - Specificity (FPR)")
    axs[0, 2].set_xlim([-0.05, 1.05])
    axs[0, 2].set_ylim([-0.05, 1.05])
    axs[0, 2].set_ylabel("Sensitivity (TPR)")
    axs[0, 2].scatter(train_report.fpr, train_report.tpr, s=1., c="red", label="train")
    axs[0, 2].scatter(eval_report.fpr, eval_report.tpr, marker="*", s=10.,
                      c="blue", label="eval")
    axs[0, 2].plot([0., 1.], [0., 1.], "-k", linewidth=0.5)
    axs[0, 2].legend()
    # plot learning rate
    axs[1, 0].set_title("Learning Rate")
    axs[1, 0].set_xlabel("Block")
    axs[1, 0].set_ylabel("Learning Rate")
    axs[1, 0].plot(train_report.block, train_report.learning_rate, "-r", linewidth=1.)
    #axs[1, 0].plot(train_epoch_stats.index, train_epoch_stats.learning_rate, "-r", linewidth=1.)
    # axs[1, 0].plot(epoch_ticks, lr_ma, "-b")
    # plot epoch stats
    axs[1, 1].set_title("F1 Scores")
    axs[1, 1].set_xlabel("Epoch")
    axs[1, 1].set_ylabel("Score")
    axs[1, 1].set_ylim([-0.05, 1.05])
    axs[1, 1].plot(epoch_ticks, eval_report.cancer_f1, "-r",
                   linewidth=1.5, label="eval_cancer_f1")
    axs[1, 1].plot(epoch_ticks, eval_report.lat_f1, "-b",
                   linewidth=1.5, label="eval_lat_f1")
    axs[1, 1].plot(epoch_ticks, eval_report.view_f1, "-g",
                   linewidth=1.5, label="eval_view_f1")
    axs[1, 1].plot(epoch_ticks, eval_report.implant_f1, "-y",
                   linewidth=1.5, label="eval_imp_f1")
    axs[1, 1].plot(epoch_ticks, balf1scores, ".r",
                   linewidth=1.5, label="bal_cancer_eval_f1")
    axs[1, 1].plot(epoch_ticks, balf1latscores, ".b",
                   linewidth=1.5, label="bal_lat_eval_f1")
    axs[1, 1].plot(epoch_ticks, train_epoch_stats.cancer_f1[:lim], "--r",
                   linewidth=1.5, label="train_cancer_f1")
    axs[1, 1].plot(epoch_ticks, train_epoch_stats.lat_f1[:lim], "--b",
                   linewidth=1.5, label="train_lat_f1")

    axs[1, 1].legend()
    # plot Recall, Precision and TNR
    axs[1, 2].set_title("Other Statistics")
    axs[1, 2].set_xlabel("Epoch")
    axs[1, 2].set_ylabel("Score")
    axs[1, 2].set_ylim([-0.05, 1.05])
    axs[1, 2].plot(epoch_ticks, eval_report.tpr, "-g",
                   linewidth=1.5, label="eval_tpr")
    axs[1, 2].plot(epoch_ticks, train_epoch_stats.tpr[:lim], "--g",
                   linewidth=1., label="train_tpr")
    axs[1, 2].plot(epoch_ticks, eval_report.tnr, "-y",
                   linewidth=1.5, label="eval_tnr")
    axs[1, 2].plot(epoch_ticks, train_epoch_stats.tnr[:lim], "--y",
                   linewidth=1., label="train_tnr")
    axs[1, 2].legend()
    plt.tight_layout()
    plt.show() 


In [ ]:
train_report_path = "/home/isaiah/kaggle/mammo/results/20230227/densenet_train_rep_01.csv"
val_report_path = "/home/isaiah/kaggle/mammo/results/20230227/densenet_eval_rep_01.csv"
window_size = 50
trep = pd.read_csv(train_report_path)
vrep = pd.read_csv(val_report_path)
Dashboard2(trep, vrep, window_size)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to("cpu")

epochs = 100
optimizer = Adam(model.parameters(), lr=3.)
step_size = 1000
scheduler = CyclicLR(optimizer, base_lr=1., max_lr=10., 
                     step_size_up=step_size, mode="triangular2", 
                     last_epoch=1000, cycle_momentum=False)
for _ in range(step_size):
    scheduler.step()   
lrs = []
for epoch in range(1, epochs + 1):
    lrs.append(scheduler.get_last_lr())
    scheduler.step()
fig3 = plt.figure()
ax3 = plt.axes()
ax3.plot(list(range(1, epochs + 1)), lrs)
fig3.show()